In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import pandas as pd
import dataset as wsd
import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

BATCH_SIZE = 16
EMBED_DIM = 512
TRANSFORMER_EMBED = 768
IMAGE_SIZE = 255

class Projection(nn.Module):
    def __init__(self, d_in: int, d_out: int, p: float=0.5) -> None:
        super().__init__()
        self.linear1 = nn.Linear(d_in, d_out, bias=False)
        self.linear2 = nn.Linear(d_out, d_out, bias=False)
        self.layer_norm = nn.LayerNorm(d_out)
        self.drop = nn.Dropout(p)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        embed1 = self.linear1(x)
        embed2 = self.drop(self.linear2(F.gelu(embed1)))
        embeds = self.layer_norm(embed1 + embed2)
        return embeds

In [18]:
class VisionEncoder(nn.Module):
    def __init__(self, d_out: int) -> None:
        super().__init__()
        base = models.resnet34(pretrained=True)
        d_in = base.fc.in_features
        base.fc = nn.Identity()
        self.base = base
        self.projection = Projection(d_in, d_out)
        for p in self.base.parameters():
            p.requires_grad = False

    def forward(self, x):
        projected_vec = self.projection(self.base(x))
        projection_len = torch.norm(projected_vec, dim=-1, keepdim=True)
        return projected_vec / projection_len

In [19]:
class TextEncoder(nn.Module):
    def __init__(self, d_out: int) -> None:
        super().__init__()
        self.base = AutoModel.from_pretrained("distilbert-base-multilingual-cased")
        self.projection = Projection(TRANSFORMER_EMBED, d_out)
        for p in self.base.parameters():
            p.requires_grad = False

    def forward(self, x):
        out = self.base(x)[0]
        out = out[:, 0, :]  # get CLS token output
        projected_vec = self.projection(out)
        projection_len = torch.norm(projected_vec, dim=-1, keepdim=True)
        return projected_vec / projection_len

In [20]:
class Tokenizer:
    def __init__(self, tokenizer: BertTokenizer) -> None:
        self.tokenizer = tokenizer

    def __call__(self, x: str) -> AutoTokenizer:
        return self.tokenizer(
            x, truncation=True, padding=True, return_tensors="pt"
        )

In [21]:
def metrics(similarity: torch.Tensor):
    y = torch.arange(len(similarity)).to(similarity.device)
    img2cap_match_idx = similarity.argmax(dim=1)
    cap2img_match_idx = similarity.argmax(dim=0)

    img_acc = (img2cap_match_idx == y).float().mean()
    cap_acc = (cap2img_match_idx == y).float().mean()

    return img_acc, cap_acc

In [62]:
class CustomModel(nn.Module):
    def __init__(self, lr: float = 1e-3) -> None:
        super().__init__()
        self.vision_encoder = VisionEncoder(EMBED_DIM)
        self.caption_encoder = TextEncoder(EMBED_DIM)
        self.tokenizer = Tokenizer(AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased"))
        self.lr = lr
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def forward(self, images, text):
        text = self.tokenizer(text).to(self.device)

        image_embed = self.vision_encoder(images)
        caption_embed = self.caption_encoder(text["input_ids"])

        similarity = caption_embed @ image_embed.T

        loss = self.CLIP_loss(similarity)
        img_acc, cap_acc = metrics(similarity)

        return loss, img_acc, cap_acc
    
    def CLIP_loss(self, logits: torch.Tensor) -> torch.Tensor:
        n = logits.shape[1]      # number of samples
        labels = torch.arange(n).to(self.device) # Create labels tensor
        # Calculate cross entropy losses along axis 0 and 1
        loss_i = F.cross_entropy(logits.transpose(0, 1), labels, reduction="mean")
        loss_t = F.cross_entropy(logits, labels, reduction="mean")
        # Calculate the final loss
        loss = (loss_i + loss_t) / 2

        return loss
    
    def top_image(self, images, text):
        text = self.tokenizer(text).to(self.device)
        caption_embed = self.caption_encoder(text["input_ids"])

        similarities = []

        for image in images:
            image_embed = self.vision_encoder(image.to(self.device))
            similarities.append(F.cosine_similarity(image_embed, caption_embed, dim=1).item())

        #top_image = np.argsort(similarities)[-1:][::-1]

        return similarities

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)

/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
model.device

'cuda'

In [35]:
optimizer = torch.optim.Adam([
    {'params': model.vision_encoder.parameters()},
    {'params': model.caption_encoder.parameters()}
], lr=model.lr)

In [36]:
from torchvision import transforms as tt

scale = tt.Resize((IMAGE_SIZE, IMAGE_SIZE))
tensor = tt.ToTensor()
image_composed = tt.transforms.Compose([scale, tensor])

In [37]:
train_set = wsd.VisualWSDDataset(mode="train", image_transform=image_composed)
test_set = wsd.VisualWSDDataset(mode="test", image_transform=image_composed, test_lang='en')

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [38]:
start_epoch = 0
num_epochs = 1

batch_zero = True
for epoch in range(start_epoch, num_epochs):
    model.train()
    for batch in test_loader:
        image = batch["correct_img"].to(device)
        text = batch["label_context"]
        # images, text = batch
        loss, img_acc, cap_acc = model(image, text)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_zero:
          print(f"Epoch [{0}/{num_epochs}], Batch Loss: {loss.item()}")
          batch_zero = False


    # Print training statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], Batch Loss: {loss.item()}")

print("Training complete.")

Epoch [0/1], Batch Loss: 3.4619855880737305


/home/ruwen/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/1], Batch Loss: 2.6944098472595215
Training complete.


In [39]:
from datetime import datetime
stamp = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')

path = "./results/" + stamp
torch.save(model.state_dict(), path)

In [40]:

# if k is 1 gives all instances with the correct prediction as top prediction
# if k > 1 the correct prediction is in the top k predictions of the model
def hit(results, k):
    counter = 0

    for r in results:
        sims = np.absolute(r[1])
        sorted = np.argsort(sims)[:k]

        if r[0] in sorted:
            counter += 1

    return counter / len(results)

def mrr(results):
    sum = 0

    for r in results:
        sims = np.absolute(r[1])
        sorted = np.argsort(sims)
        sum += 1/(np.where(sorted==r[0])[0][0]+1)

    return sum / len(results)

In [63]:
eval_model = CustomModel().to(device)
model_name = "2024-06-12-15:44:12"
eval_model.load_state_dict(torch.load("./results/" + model_name))
eval_model.eval()

/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomModel(
  (vision_encoder): VisionEncoder(
    (base): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (b

In [90]:
def test_loop(loader):
    results = []

    for i,batch in enumerate(test_loader):
        images = batch["imgs"]
        text = batch["label_context"]
        correct_idx = batch["correct_idx"].item()
        sims = eval_model.top_image(images, text)

        results.append((correct_idx, sims))
        top_image = np.argsort(np.absolute(sims))[0]

        #print(np.argsort(np.absolute(sims)))
        print("batch: " + str(i+1) + "/" + str(len(test_loader)) + " predicted: " + str(top_image) + " correct: " + str(correct_idx))

        if i == 50:
            break

    return results

In [91]:
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

results = test_loop(test_loader)


[0 7 3 1 9 6 5 2 8 4]
batch: 1/463 predicted: 0 correct: 8
[3 7 4 0 5 2 9 6 1 8]
batch: 2/463 predicted: 3 correct: 0
[8 4 0 7 9 3 1 6 2 5]
batch: 3/463 predicted: 8 correct: 5
[7 5 0 3 1 2 8 6 9 4]
batch: 4/463 predicted: 7 correct: 6
[8 3 0 1 6 2 7 4 5 9]
batch: 5/463 predicted: 8 correct: 2
[9 5 0 2 3 8 6 4 7 1]
batch: 6/463 predicted: 9 correct: 6
[7 9 2 5 3 8 4 1 6 0]
batch: 7/463 predicted: 7 correct: 9
[2 6 7 1 0 3 9 4 8 5]
batch: 8/463 predicted: 2 correct: 7
[9 8 0 4 6 5 1 7 3 2]
batch: 9/463 predicted: 9 correct: 2
[5 8 3 6 9 4 0 7 2 1]
batch: 10/463 predicted: 5 correct: 9
[3 2 5 1 0 8 4 7 6 9]
batch: 11/463 predicted: 3 correct: 4
[0 2 3 4 5 6 1 9 8 7]
batch: 12/463 predicted: 0 correct: 2
[9 5 0 2 8 3 6 1 4 7]
batch: 13/463 predicted: 9 correct: 5
[0 6 2 1 4 3 8 7 9 5]
batch: 14/463 predicted: 0 correct: 6
[3 1 8 5 0 2 4 6 9 7]
batch: 15/463 predicted: 3 correct: 4
[5 2 8 7 9 0 1 6 4 3]
batch: 16/463 predicted: 5 correct: 7
[0 5 3 8 4 7 9 2 6 1]
batch: 17/463 predicted: 0 

In [93]:
print("hit@1: " + str(hit(results, 1)))
print("mrr: " + str(mrr(results)))

hit@1: 0.11764705882352941
mrr: 0.31030967942732646


In [12]:
from transformers import DistilBertTokenizerFast

bert_tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokens(texts):
    return bert_tokenizer(texts, truncation=True, padding=True)


In [13]:
train_set_token = wsd.VisualWSDDataset(mode="train", image_transform=image_composed, tokenizer=tokens)
trial_set_token = wsd.VisualWSDDataset(mode="val", image_transform=image_composed, tokenizer=tokens)

In [14]:
trial_set_token[0]

/home/ruwen/ki-lab-ss23-MindCraft/dataset.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['images'] = torch.tensor(correct_image).detach()


{'input_ids': tensor([  101,  1998, 21716, 11960,  3392,   102,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0]),
 'images': tensor([[[0.0510, 0.0118, 0.0510,  ..., 0.3059, 0.3059, 0.3216],
          [0.1059, 0.0431, 0.0588,  ..., 0.2510, 0.2471, 0.2667],
          [0.0667, 0.0510, 0.0784,  ..., 0.1490, 0.1294, 0.1804],
          ...,
          [0.6471, 0.5412, 0.4784,  ..., 0.1255, 0.1333, 0.1882],
          [0.5922, 0.4941, 0.4706,  ..., 0.1569, 0.1412, 0.2275],
          [0.5725, 0.5059, 0.4941,  ..., 0.1608, 0.1725, 0.3255]],
 
         [[0.1059, 0.0863, 0.0706,  ..., 0.3176, 0.3294, 0.3451],
          [0.1294, 0.0980, 0.0588,  ..., 0.2510, 0.2627, 0.2941],
          [0.1333, 0.0980, 0.0431,  ..., 0.1451, 0.1490, 0.2039],
          ...,
          [0.6627, 0.5529, 0.4667,  ..., 0.2706, 0.3059, 0.3020],
          [0.6078, 0.4980, 0.4706,  ..., 0.3137, 0.2863, 0.2510],
          [0.5843, 0.5059, 0.4980,  ..., 0.3137, 0.2627, 0.2235]],
 
         [[0.0235, 0.0039, 0.

In [15]:
class CustomModelHugging(nn.Module):
    def __init__(self, lr: float = 1e-3) -> None:
        super().__init__()
        self.vision_encoder = VisionEncoder(EMBED_DIM)
        self.caption_encoder = TextEncoder(EMBED_DIM)
        #self.tokenizer = Tokenizer(AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased"))
        self.lr = lr
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def forward(self, images, input_ids, attention_mask):
        image_embed = self.vision_encoder(images)
        caption_embed = self.caption_encoder(input_ids)

        similarity = caption_embed @ image_embed.T

        loss = self.CLIP_loss(similarity)
        img_acc, cap_acc = metrics(similarity)

        return loss, img_acc, cap_acc
    
    def CLIP_loss(self, logits: torch.Tensor) -> torch.Tensor:
        n = logits.shape[1]      # number of samples
        labels = torch.arange(n).to(self.device) # Create labels tensor
        # Calculate cross entropy losses along axis 0 and 1
        loss_i = F.cross_entropy(logits.transpose(0, 1), labels, reduction="mean")
        loss_t = F.cross_entropy(logits, labels, reduction="mean")
        # Calculate the final loss
        loss = (loss_i + loss_t) / 2

        return loss

In [16]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=300,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    dataloader_num_workers=3,
)

model = CustomModelHugging()

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_set_token,         # training dataset
    eval_dataset=trial_set_token,             # evaluation dataset
    tokenizer=bert_tokenizer,
    optimizers=(optimizer, None)        # no scheduler
)

trainer.train()

/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ruwen/ki-lab-ss23-MindCraft/dataset.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['images'] = torch.tensor(correct_image).detach()
/home/ruwen/ki

Step,Training Loss
10,4.159000
20,4.158900
30,4.159400
40,4.159100
50,4.159100
60,4.159100
70,4.159300
80,4.158100
90,4.159600
100,4.158800


/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (102080000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (141880000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (178315136 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python3.10/site-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (148635000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ruwen/miniconda3/envs/mindcraft_wsd/lib/python

TrainOutput(global_step=606, training_loss=4.146454820538511, metrics={'train_runtime': 6095.2045, 'train_samples_per_second': 6.334, 'train_steps_per_second': 0.099, 'total_flos': 0.0, 'train_loss': 4.146454820538511, 'epoch': 3.0})

In [ ]:
# TODO
# Trainer ausprobieren
# DataLoader anpassen mit Validation Dataset